In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.9 MB/s eta 0:00:00

In [2]:
import pandas as pd
data = pd.read_csv("/content/bertnertrainfinal.csv",encoding="latin1")

In [3]:
data =data.fillna(method ="ffill")

<ipython-input-3-9136542f6b6f>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data =data.fillna(method ="ffill")


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [6]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [7]:
X= data[["sentence_id","words"]]
Y =data["NER Label"]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2,random_state=90)

In [9]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [10]:
from simpletransformers.ner import NERModel,NERArgs

In [11]:
label = data["NER Label"].unique().tolist()
label

['B-Name',
 'I-Name',
 'B-Designation',
 'I-Designation',
 'O',
 'B-Companies worked at',
 'I-Companies worked at',
 'B-Location',
 'B-Quality',
 'I-Quality',
 'B-Years of Experience',
 'I-Years of Experience',
 'B-Skills',
 'I-Skills',
 'NER Label',
 'I-Location',
 'B-Degree',
 'I-Degree',
 'B-College Name',
 'I-College Name',
 'B-Graduation Year',
 'isid=rex-download&ikw=download-top&co=IN',
 'I-Graduation Year',
 'B-Project Name',
 'B-Team',
 'I-Team',
 'I-Project Name',
 'B-Companies worked',
 'B-Certifications',
 'I-Certifications',
 'BSc(MECs) 2nd year',
 'new',
 'N-Name',
 'vendor',
 'meet',
 'working',
 'B-Skill',
 'M',
 'in',
 'demonstrating',
 'B-Projects',
 'I-Years of',
 'B-S',
 'I-Projects',
 'B-Tools',
 'B-Email',
 'B-URL',
 'B-Methodology',
 'I-Methodology',
 'B-Section',
 'B-Action',
 'B-Field',
 'B-Dates',
 'I-Dates',
 'I-Field',
 'B-Quantity',
 '0',
 'I-Quantity',
 '000+',
 'I-Tools',
 'I-Email']

In [12]:
train_data

,sentence_id,words,labels
12229,2592,March,B-Years of Experience
18930,26,Engineer,I-Designation
17933,849,related,O
1338,380,Gathering,I-Skills
16499,225,linkedin.com,B-URL
...,...,...,...
9539,2481,every,B-Quality
6815,559,Manage,B-Skills
5746,2,Karnataka,I-Location
2717,1914,and,O


In [13]:
args = NERArgs()
args.num_train_epochs =19
args.learning_rate = 4.464733792261065e-05
args.train_batch_size = 16
args.eval_batch_size = 8
args.gradient_accumulation_steps = 16
args.warmup_ratio = 0.08876847289936485
args.weight_decay =  0.00023027403760175153
args.evaluation_strategy = "steps"
args.eval_steps = 100
args.logging_steps = 50
args.save_strategy = "steps"
args.save_steps = 500
args.load_best_model_at_end = True
args.early_stopping_patience = 3
args.early_stopping_threshold = 0.01
args.overwrite_output_dir = True
args.dropout=0.40358622263304433




In [14]:
!rm -rf /kaggle/working/*


In [15]:
model = NERModel('bert', 'bert-large-cased',labels=label,args =args)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [16]:
model.args.save_model_every_epoch = False  # Save only after the final epoch


In [17]:
model.args.tensorboard_dir = None


In [18]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 19:   0%|          | 0/161 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:782: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Epoch 16 of 19:   0%|          | 0/161 [00:00<?, ?it/s]

Running Epoch 17 of 19:   0%|          | 0/161 [00:00<?, ?it/s]

Running Epoch 18 of 19:   0%|          | 0/161 [00:00<?, ?it/s]

Running Epoch 19 of 19:   0%|          | 0/161 [00:00<?, ?it/s]

(190, 1.0792840998754591)

In [19]:
model.model.save_pretrained("./ner_bert_model")


model.tokenizer.save_pretrained("./ner_bert_model")

('./ner_bert_model/tokenizer_config.json',
 './ner_bert_model/special_tokens_map.json',
 './ner_bert_model/vocab.txt',
 './ner_bert_model/added_tokens.json')

In [20]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/102 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NER Label seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: new seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [12]:
!pip install PyMUPDF
import fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.1 MB/s eta 0:00:00


In [13]:
import spacy
nlp=spacy.load("en_core_web_sm")

def pre(yo):
 doc=nlp(yo)

 string=[]

 for token in doc:
  if token.is_stop==False and token.is_punct==False:
     string.append(token.lemma_)

 return " ".join(string)

In [14]:
def preprocess_final(name):
  pdf_file = name
  doc = fitz.open(pdf_file)

  text = " "

  for page_num in range(doc.page_count):
      page = doc.load_page(page_num)
      text += page.get_text()

  return (" ".join(text.split("\n")))

In [41]:
yo1=pre(preprocess_final("/content/data_analyst.pdf"))

In [48]:
yo1

'  Payton Webster Data Analyst | Data Visualization | SQL Expert help@enhancv.com linkedin.com Cleveland OH objective Eager apply datum analysis skill dynamic environment skilled   SQL Python datum visualization aspire contribute data drive   decision make process eager develop data analytic EXPERIENCE Data Analysis Intern IBM 2023 2023 \xa0  Cleveland oh internship focus datum analysis visualization analyze large dataset SQL Python identify key trend   pattern improve data drive decision making 30 develop present data visualization Tableau result   enhance stakeholder understanding complex data set collaborate project streamline data collection process   reduce datum processing time 25 Business Intelligence Intern Accenture 2023 2023 \xa0  Cleveland oh internship focus business intelligence datum management assist development automate reporting system   Python increase report generation efficiency 40 conduct depth datum analysis support business strategy   development contribute 15 in

In [42]:
yo2=(preprocess_final("/content/game_animator.pdf"))

In [43]:
yo3=(preprocess_final("/content/data_analyst2.pdf"))

In [44]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from typing import List, Dict
from collections import Counter

class EnsembleNER:
    def initialize(self, pretrained_model_path, device='cuda'):

        self.bert_model = AutoModelForTokenClassification.from_pretrained("/content/ner_bert_model")
        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-large-cased')

        # Device setup
        self.device = device
        self.bert_model.to(device)
        self.bert_model.eval()

    def predict(self, text: str) -> List[Dict[str, str]]:
        # Tokenize and get predictions from the BERT model
        bert_tokens = self.bert_tokenizer(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,
            padding=True,
            truncation=True
        ).to(self.device)


        offset_mapping = bert_tokens.pop('offset_mapping')

        with torch.no_grad():
            bert_outputs = self.bert_model(**bert_tokens)
            bert_predictions = torch.argmax(bert_outputs.logits, dim=2)[0]


        word_predictions = []
        current_word = ""
        current_labels = []

        input_ids = bert_tokens["input_ids"][0]

        for idx, (start, end) in enumerate(offset_mapping[0]):
            if start == 0 and end == 0:
                continue


            token_text = self.bert_tokenizer.decode(input_ids[idx].item())
            pred_label = self.bert_model.config.id2label[bert_predictions[idx].item()]


            if token_text.startswith("##"):
                current_word += token_text[2:]
                current_labels.append(pred_label)
            else:
                if current_word:

                    final_label = Counter(current_labels).most_common(1)[0][0]
                    word_predictions.append({"word": current_word, "label": final_label})


                current_word = token_text
                current_labels = [pred_label]

        if current_word and current_labels:
            final_label = Counter(current_labels).most_common(1)[0][0]
            word_predictions.append({"word": current_word, "label": final_label})

        return word_predictions

In [52]:


    ensemble = EnsembleNER(
        pretrained_model_path="/content/ner_bert_model",
        device="cuda" if torch.cuda.is_available() else "cpu")



    text =yo1


    predictions = ensemble.predict(text)


    for pred in predictions:
        print(pred)






{'word': 'Payton', 'label': 'B-Skills'}
{'word': 'Webster', 'label': 'B-Skills'}
{'word': 'Data', 'label': 'B-Skills'}
{'word': 'Analyst', 'label': 'I-Designation'}
{'word': '|', 'label': 'O'}
{'word': 'Data', 'label': 'B-Skills'}
{'word': 'Visualization', 'label': 'B-Skills'}
{'word': '|', 'label': 'O'}
{'word': 'SQL', 'label': 'B-Skills'}
{'word': 'Expert', 'label': 'B-Skills'}
{'word': 'help', 'label': 'B-Skills'}
{'word': '@', 'label': 'O'}
{'word': 'enhancv', 'label': 'B-Skills'}
{'word': '.', 'label': 'O'}
{'word': 'com', 'label': 'I-Companies worked at'}
{'word': 'linkedin', 'label': 'B-Skills'}
{'word': '.', 'label': 'O'}
{'word': 'com', 'label': 'B-Skills'}
{'word': 'Cleveland', 'label': 'B-Location'}
{'word': 'OH', 'label': 'I-Location'}
{'word': 'objective', 'label': 'B-Skills'}
{'word': 'Eager', 'label': 'B-Skills'}
{'word': 'apply', 'label': 'B-Skills'}
{'word': 'datum', 'label': 'B-Skills'}
{'word': 'analysis', 'label': 'B-Skills'}
{'word': 'skill', 'label': 'O'}
{'word':

In [53]:
from collections import defaultdict
from typing import List, Dict

def group_entities_unique(predictions: List[Dict[str, str]]) -> Dict[str, list]:
    entities = defaultdict(set)
    current_entity = []
    current_label = None

    for pred in predictions:
        word = pred['word']
        label = pred['label']

        if label.startswith("B-"):
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))
            current_label = label[2:]
            current_entity = [word]
        elif label.startswith("I-") and current_label == label[2:]:
            current_entity.append(word)
        else:
            if current_entity and current_label:
                entities[current_label].add(" ".join(current_entity))
            current_entity = []
            current_label = None

    if current_entity and current_label:
        entities[current_label].add(" ".join(current_entity))

    # Convert sets back to lists for JSON
    return {label: list(entity_set) for label, entity_set in entities.items()}



grouped_output = group_entities_unique(predictions)

# Print the result
print(grouped_output)


{'Skills': ['report', 'Volunteered', 'Learning', 'cover', 'transform', 'excel', 'b', 'commit', 'Storytelling', 'set', 'demonstrate', 'make', 'trend', 'generation', 'analyze', 'Bank', 'Generation', 'CERTIFICATION', 'decision', 'dashboard', 'understandable', 'datum', 'format', 'Collaboration', 'q', 'Payton', 'development', 'Report', 'participate', 'processing', 'aspire', 'objective', 'story', 'Expert', 'datum analysis', 'SQL', 'data', 'Hands', 'operational', 'Skilled', 'Food', 'drive', 'enhancv', 'stakeholder', 'LANGUAGES', 'English', 'visualization', 'team', 'develop', 'manipulation', 'management', 'Training', 'making', 'workshop', 'Spanish', 'training', 'depth', 'optimize', 'intelligence', 'Business', 'Webster', 'Eager', 'linkedin', 'creation', 'query', 'feedback', 'allocation', 'insight', 'analysis', 'automate', 'Tableau', 'pattern', 'streamline', 'Python', 'provide', 'project', 'strategy', 'system', 'evidence', 'collaborate', 'reporting', 'understanding', 'focus', 'Data', 'resource',